In [1]:
import shelve
from time import sleep
import vk_api
from vk_api import VkUpload
from vk_api.longpoll import VkLongPoll, VkEventType
import requests
from vk_api.execute import VkFunction
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import json
import datetime
import numpy as np

In [2]:
start_comm_id = 20900000
comm_N = 10000

In [3]:
vk_func_getMembQuan = VkFunction(args=('x'), code='''
    var memb = [];
    var i = 0;
    while (i < 25) 
    {
        memb.push(API.groups.getMembers({"group_id": i + %(x)s})["count"]);
        i = i + 1;
    };
    return memb;
''')

vk_func_getMembIDList = VkFunction(args=('x', 'y'), code='''
    var offset = %(y)s;
    var IDList = [];
    var i = 0;
    while (i < 25)
    {
        IDList.push(API.groups.getMembers({"group_id": %(x)s, "offset" : offset + i * 1000})["items"]);
        i = i + 1;
    }
    return IDList;
''')

In [6]:
vk_session = vk_api.VkApi(token='d082c8543db0fb23abd5ba5e8e8ef60dee208a5e5373068b4e7ebdb37d46ec61a31474d57f9e99481a403') 
try:
    vk_session.auth(reauth=True, token_only=True)
except vk_api.AuthError as error_msg:
    print(error_msg)

Login is required to auth


In [9]:
vk = vk_session.get_api()
#vk.RPS_DELAY = 0.05
print('--start--')

MQData = []
for comm_id in range(start_comm_id, start_comm_id + comm_N, 25):
    MQData.append(vk_func_getMembQuan(vk, comm_id))
    if (comm_id // 1000 is 0):
        print(comm_id - start_comm_id)

AttributeError: 'VkApiMethod' object has no attribute 'RPS_DELAY'

In [8]:
MQData = [y for x in MQData for y in x]
print(MQData)

[13, 1, 1, 1, 2, 1, 16, 2, 1, 2306, 1, 142, 1, 1, 0, 0, None, 120, 170, 0, 81, 4, 0, 1, 1, 7, 3276, None, None, 16, 30, 12, 20, 1, None, 0, 2, 1, 34, 0, 11, 4, 196, 49, 1, 106, 469, 2, 1558, 3, 409, 31, 24, 2, 2, 4, 1, 0, 13, 4, 463, None, 0, 2, 302, None, 36, 61, None, 50, 5, 1, 239, 7, 94, 3, 6, 3, 1, 1, 865, 251, 2, 45, 8, 63, 0, None, 3, 2, 28, 58, 6, 0, 9, 1, 1, 1, 1, 1, 2, 15, 46, 2, 0, 1177, 184, 6, 16, 559, 14, 4, 0, 219, 5, 4, 0, 5, 3, 19, 203, 119, 39, None, 1, 10, None, 5, None, 0, 285, 38, 5, 7, 9, 0, 1, 1, 33, 0, 15, 1, None, 1, 94, None, 2, 20, None, 16, 4, 0, 1, None, 8, 223, 6095, 0, 14, 0, 1, 3, 0, 322, 2, None, 2, 1, 0, 1, 1, 2, 0, 0, 0, 3, 0, None, 11, 12, 3, 12, 39, 18, 6, 24, 11, 3, 0, 84, None, None, 8630, 0, 4, 3, 0, 13, 1, 4, 0, 11, None, 2, 0, 11, 107, 119, 122, 2, 40, 2, 2, 265, 1, 690, 0, 1, None, 300, 34, 0, 2, 0, 32, 2027, 2, 0, 1, 0, 112, 494, 2, 45, 411, 8, 4, 673, 1, 0, 2, 344, 301, 0, 1, 0, 6, 29, None, 2, 13, 1, 8, 2, 39, 1, 17, 179, 8, 146, 1, None, 1

In [ ]:
MQDict = {}
for i in range(comm_N):
    if (MQData[i] is not None) and (MQData[i] > 1e4):
        MQDict[start_comm_id + i] = MQData[i]
print(MQDict)

In [ ]:
IDDict = {}
for id, quan in MQDict.items():
    IDDict[id] = []
    for offset in range(0, quan, 25000):
        IDDict[id].append(vk_func_getMembIDList(vk, x=id, y=offset))
        
for id, ilist in IDDict.items():
    IDDict[id] = [z for x in IDDict[id] for y in x for z in y]

In [ ]:
np.save('IDDict.npy', IDDict) 

In [ ]:
for key, val in IDDict.items():
    print(f"{key}: {val[0:10000]}")